Ce notebook est un premier pas avant transformation en application.
Il nous permet de tester le (faible) niveau d'accessibilité de ce format notebook pour les personnes aveugles.

Quelles règles apprises : 
- Le notebook n'est globalement pas très accessible
- On peut quand même améliorer les choses en choisissant une langue source (pour faciliter le travail de l'aide audio de la personne aveugle) et en mettant une structure en titres/sous-titre facilitant la navigation.

## Preprocessing des données

In [1]:
import pandas as pd
import folium

Hyperparamètres

In [2]:
cols_access = ['entree_balise_sonore'] # Variable d'accessibilité visée
code_com_cible = "85294" # Commune d'intérêt

Download data acceslibre

In [3]:
url_access_libre = "https://www.data.gouv.fr/fr/datasets/r/5b0f44f2-e6ea-4a58-874d-6fe364b40342"
df_accesslibre = pd.read_csv(url_access_libre)

/tmp/ipykernel_84462/489921021.py:2: DtypeWarning: Columns (56,67,68,69,70,71,72,73,74,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accesslibre = pd.read_csv(url_access_libre)


In [4]:
df_accesslibre.columns

Index(['id', 'name', 'postal_code', 'commune', 'numero', 'voie', 'lieu_dit',
       'code_insee', 'siret', 'activite', 'contact_url', 'site_internet',
       'longitude', 'latitude', 'transport_station_presence',
       'stationnement_presence', 'stationnement_pmr',
       'stationnement_ext_presence', 'stationnement_ext_pmr',
       'cheminement_ext_presence', 'cheminement_ext_terrain_stable',
       'cheminement_ext_plain_pied', 'cheminement_ext_ascenseur',
       'cheminement_ext_nombre_marches', 'cheminement_ext_reperage_marches',
       'cheminement_ext_sens_marches', 'cheminement_ext_main_courante',
       'cheminement_ext_rampe', 'cheminement_ext_pente_presence',
       'cheminement_ext_pente_degre_difficulte',
       'cheminement_ext_pente_longueur', 'cheminement_ext_devers',
       'cheminement_ext_bande_guidage', 'cheminement_ext_retrecissement',
       'entree_reperage', 'entree_vitree', 'entree_vitree_vitrophanie',
       'entree_plain_pied', 'entree_ascenseur', 'entree_mar

Columns selection

In [5]:
cols_base = ['id', 'name', 'code_insee', 'postal_code', 'commune', 'numero', 'voie', 'lieu_dit', 'longitude', 'latitude', ]
df_balises = df_accesslibre[cols_base + cols_access]


Suppression des établissements sans information sur les balises sonores

In [6]:
df_balises = df_balises[df_balises['entree_balise_sonore'].notna()]
print(df_balises.shape)

(65954, 11)


In [7]:
df_balises['entree_balise_sonore'].value_counts()

entree_balise_sonore
False    63625
True      2329
Name: count, dtype: int64

In [8]:
df_balises

,id,name,code_insee,postal_code,commune,numero,voie,lieu_dit,longitude,latitude,entree_balise_sonore
4,c7d781c0-12cf-4d66-aeef-571d8d4efd4f,0'mok@bar,75118,75018,Paris,43,Rue de la Chapelle,NaN,2.359299,48.893598,False
5,893019fe-cf9e-405e-9423-91955766cb45,0RPI,76322,76120,Le Grand-Quevilly,10,Avenue John Fitzgerald Kennedy,NaN,1.050465,49.408801,False
6,983f2d7a-3056-4a2a-b5b5-2b8ff0b40234,0zo,NaN,92400,Courbevoie,40,Rue De Bezons,NaN,2.248571,48.896493,False
9,e58f3350-8ae9-4f95-8343-98b2d22cd070,1000 BORNES,14220,14800,Deauville,59,Avenue de la République,NaN,0.075918,49.357996,False
11,128b173b-71c1-4e86-bf70-91122cee51ec,1000 Club,88109,88120,Cleurie,1,Chemin de Chevrimont,NaN,6.676431,48.051255,False
...,...,...,...,...,...,...,...,...,...,...,...
526299,c6cd1343-3c94-4b95-84c5-be67b1a15fda,ZXSBN6DD3VZBGSY,NaN,75002,Paris,5,Rue Notre-Dame de Bonne Nouvelle,NaN,2.349630,48.869574,False
526304,2ae3f50d-3ecb-4a82-b677-60f0661b58f8,zyke piscine,NaN,33600,Pessac,54,Avenue Jean Jaurès,NaN,-0.626179,44.807678,False
526305,0c76f896-72a0-484c-96de-92d9349be617,Zyke Piscine,NaN,16430,Champniers,203,Rue de l’Arêtier,NaN,0.180574,45.692716,False
526306,e61427a8-da8e-474f-9fa7-e9403ac75de5,Zyke-Piscine Internationale,NaN,47240,Lafox,NaN,NaN,Lieu-dit Devant Le Moulin RN 113,0.694818,44.170681,False


Cleaning des positions GPS

In [9]:
# Convertir les colonnes latitude et longitude si nécessaire
df_balises['longitude'] = pd.to_numeric(df_balises['longitude'], errors='coerce')
df_balises['latitude'] = pd.to_numeric(df_balises['latitude'], errors='coerce')

# Supprimer les lignes avec des coordonnées invalides
df_balises = df_balises.dropna(subset=['longitude', 'latitude'])


On ne garde que les établissements d'une communes. 

In [10]:
df_balises_com = df_balises[df_balises["code_insee"]==code_com_cible]


## Résultats

### Tableau des établissements répertoriés dans Accès Libre à La Tranche sur mer

In [11]:
df_balises_com[["name","entree_balise_sonore","numero","voie", "commune"]]

,name,entree_balise_sonore,numero,voie,commune
119311,Cinéma Agnès Varda,False,NaN,Boulevard de la petite hollande,La Tranche-sur-Mer
206731,Gendarmerie - Brigade de La-Tranche-sur-Mer,False,115,Avenue Maurice Samson,La Tranche-sur-Mer
263928,La Poste,False,35,Rue victor hugo,La Tranche-sur-Mer
334914,Mairie de La Tranche-sur-Mer,False,8,Rue de l'hôtel de ville,La Tranche-sur-Mer
394369,Office de Tourisme de la Tranche sur Mer,False,1a,Rue jules ferry,La Tranche-sur-Mer


In [12]:
nb_etab_equipes = df_balises_com["entree_balise_sonore"].sum()
if (nb_etab_equipes>0):
    print(f"{nb_etab_equipes}établissements sont équipés dans la communes")
else:
    print("Ps d'établissement équipé dans la commune")

Ps d'établissement équipé dans la commune


### Carte des établissements

In [13]:
point_central = [df_balises_com['latitude'].mean(), df_balises_com['longitude'].mean()]
carte = folium.Map(location=point_central, zoom_start=6)

coordonnees = []
for _, row in df_balises_com.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<b>{row['name']}</b>",
    ).add_to(carte)
    coordonnees.append([row['latitude'], row['longitude']])

if coordonnees: 
    carte.fit_bounds(coordonnees)
display(carte)
